In [1]:
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import requests
from io import BytesIO
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
from skimage.transform import rescale, resize, downscale_local_mean
import random
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob
import random
import shutil

In [ ]:
# resize image
gts = glob.glob("Malariae/gt/*")

for gt in gts:
    print(gt)
    img = imread(gt, plugin='pil')
    new_img = resize(img,(1200,1600),anti_aliasing=True)
    new_name = gt.split(".")[0] + ".jpg"
    imsave(new_name,new_img)
    print(new_name)

print("--------img start---------") 
gts = glob.glob("Malariae/img/*")
for gt in gts:
    print(gt)
    img = imread(gt, plugin='pil')
    new_img = resize(img,(1200,1600),anti_aliasing=True)
    new_name = gt.split(".")[0] + ".jpg"
    imsave(new_name,new_img)
    print(new_name)

In [ ]:
class_id = "1"
count = 1
gts = glob.glob("Malariae/gt/*.jpg")
main_img_pth = "Malariae/img/"
for gt in gts:
    gt_image = gt.split("/")[-1]
    txt_name = "pm-{:010d}.txt".format(int(count))
    img_name = "pm-{:010d}.jpg".format(int(count))
    print(gt_image)
    img_main = imread(main_img_pth + gt_image)
    imsave(os.path.join("tmp_images/",img_name),img_main)
    img = imread(gt)
    h, w = img.shape
    thresh = threshold_otsu(img)
    bw = img > thresh
    label_image = label(bw)
    result = ""
    for region in regionprops(label_image):
        minr, minc, maxr, maxc = region.bbox
        x1 = minc / w
        y1 = minr / h
        bw = (maxc - minc) / w
        bh = (maxr - minr) / h
        xc = x1 + (bw / 2)
        yc = y1 + (bh / 2)
        result += "\n" + class_id + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh) 
    f = open("tmp_labels/" + txt_name,'w')
    result = result[1:]
    f.write(result)
    f.close()
    count += 1

In [2]:
count = 38
files = glob.glob("tmp_labels/*.txt")
degs = [45,90,135,180,225,270,315]
for file in files:
    print(file)
    f = open(file,"r")
    lines = f.readlines()
    f.close()
    rects = []
    img_name = file.split("/")[1].replace(".txt",".jpg")
    for line in lines:
        class_id, xc, yc, bw, bh = line.split(" ")
        xc = float(xc)
        yc = float(yc)
        bw = float(bw)
        bh = float(bh)
        minx = (xc - (bw/2)) * 1600
        miny = (yc - (bh/2)) * 1200
        bw = bw * 1600
        bh = bh * 1200
        maxx = minx + bw
        maxy = miny + bh
        rects.append(BoundingBox(x1=int(minx),y1=int(miny),x2=int(maxx),y2=int(maxy),label=class_id))
    img = Image.open("tmp_images/"+img_name)
    img = np.array(img)
    bbs = BoundingBoxesOnImage(rects,shape=img.shape)
    for deg in degs:
        name = "pm-{:010d}".format(count)
        image_aug, bbs_aug = iaa.Affine(rotate=deg,mode="reflect")(image=img, bounding_boxes=bbs)
        aug_h, aug_w, aug_c = image_aug.shape
        clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=False).clip_out_of_image()
        bboxs = ""
        for bb in clip_off_bbs.bounding_boxes:
            x1 = bb.x1
            y1 = bb.y1
            x2 = bb.x2
            y2 = bb.y2
            bb_w = (x2-x1)/aug_w
            bb_h = (y2-y1)/aug_h
            bb_xc = (x1/aug_w) + (bb_w/2)
            bb_yc = (y1/aug_h) + (bb_h/2)
            bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
        bboxs = bboxs[1:]
        im_aug = Image.fromarray(image_aug)
        im_aug.save(os.path.join('tmp_images',name+".jpg"))
        f = open(os.path.join('tmp_labels',name+".txt"),'w')
        f.write(bboxs)
        f.close()
        print("{} {}".format(count,deg))
        count += 1

tmp_labels/pm-0000000001.txt
38 45
39 90
40 135
41 180
42 225
43 270
44 315
tmp_labels/pm-0000000002.txt
45 45
46 90
47 135
48 180
49 225
50 270
51 315
tmp_labels/pm-0000000003.txt
52 45
53 90
54 135
55 180
56 225
57 270
58 315
tmp_labels/pm-0000000004.txt
59 45
60 90
61 135
62 180
63 225
64 270
65 315
tmp_labels/pm-0000000005.txt
66 45
67 90
68 135
69 180
70 225
71 270
72 315
tmp_labels/pm-0000000006.txt
73 45
74 90
75 135
76 180
77 225
78 270
79 315
tmp_labels/pm-0000000007.txt
80 45
81 90
82 135
83 180
84 225
85 270
86 315
tmp_labels/pm-0000000008.txt
87 45
88 90
89 135
90 180
91 225
92 270
93 315
tmp_labels/pm-0000000009.txt
94 45
95 90
96 135
97 180
98 225
99 270
100 315
tmp_labels/pm-0000000010.txt
101 45
102 90
103 135
104 180
105 225
106 270
107 315
tmp_labels/pm-0000000011.txt
108 45
109 90
110 135
111 180
112 225
113 270
114 315
tmp_labels/pm-0000000012.txt
115 45
116 90
117 135
118 180
119 225
120 270
121 315
tmp_labels/pm-0000000013.txt
122 45
123 90
124 135
125 180
126 225

In [3]:
# create test_image
images = glob.glob("tmp_images/*.jpg")
for image in images:
    img = Image.open(image)
    weigth,height = img.size
    print(weigth)
    txt_name = image.replace("tmp_images/","tmp_labels/").replace(".jpg",".txt")
    f = open(txt_name,"r")
    bboxs = list(f)
    f.close()
    for box in bboxs:
        coor = box.split()
        label = int(coor[0])
        xc = float(coor[1])*weigth
        yc = float(coor[2])*height
        bb_w = float(coor[3])*weigth
        bb_h = float(coor[4])*height
        x1 = xc-(bb_w/2)
        y1 = yc-(bb_h/2)
        x2 = x1 + bb_w
        y2 = y1 + bb_h
        draw = ImageDraw.Draw(img)
        draw.rectangle((x1, y1,x2,y2), fill=None,outline="red",width=5)
    save_path = image.replace("tmp_images/","test_images/")
    img.save(save_path)
    print(save_path)

1600
test_images/pm-0000000001.jpg
1600
test_images/pm-0000000002.jpg
1600
test_images/pm-0000000003.jpg
1600
test_images/pm-0000000004.jpg
1600
test_images/pm-0000000005.jpg
1600
test_images/pm-0000000006.jpg
1600
test_images/pm-0000000007.jpg
1600
test_images/pm-0000000008.jpg
1600
test_images/pm-0000000009.jpg
1600
test_images/pm-0000000010.jpg
1600
test_images/pm-0000000011.jpg
1600
test_images/pm-0000000012.jpg
1600
test_images/pm-0000000013.jpg
1600
test_images/pm-0000000014.jpg
1600
test_images/pm-0000000015.jpg
1600
test_images/pm-0000000016.jpg
1600
test_images/pm-0000000017.jpg
1600
test_images/pm-0000000018.jpg
1600
test_images/pm-0000000019.jpg
1600
test_images/pm-0000000020.jpg
1600
test_images/pm-0000000021.jpg
1600
test_images/pm-0000000022.jpg
1600
test_images/pm-0000000023.jpg
1600
test_images/pm-0000000024.jpg
1600
test_images/pm-0000000025.jpg
1600
test_images/pm-0000000026.jpg
1600
test_images/pm-0000000027.jpg
1600
test_images/pm-0000000028.jpg
1600
test_images/pm-

test_images/pm-0000000235.jpg
1600
test_images/pm-0000000236.jpg
1600
test_images/pm-0000000237.jpg
1600
test_images/pm-0000000238.jpg
1600
test_images/pm-0000000239.jpg
1600
test_images/pm-0000000240.jpg
1600
test_images/pm-0000000241.jpg
1600
test_images/pm-0000000242.jpg
1600
test_images/pm-0000000243.jpg
1600
test_images/pm-0000000244.jpg
1600
test_images/pm-0000000245.jpg
1600
test_images/pm-0000000246.jpg
1600
test_images/pm-0000000247.jpg
1600
test_images/pm-0000000248.jpg
1600
test_images/pm-0000000249.jpg
1600
test_images/pm-0000000250.jpg
1600
test_images/pm-0000000251.jpg
1600
test_images/pm-0000000252.jpg
1600
test_images/pm-0000000253.jpg
1600
test_images/pm-0000000254.jpg
1600
test_images/pm-0000000255.jpg
1600
test_images/pm-0000000256.jpg
1600
test_images/pm-0000000257.jpg
1600
test_images/pm-0000000258.jpg
1600
test_images/pm-0000000259.jpg
1600
test_images/pm-0000000260.jpg
1600
test_images/pm-0000000261.jpg
1600
test_images/pm-0000000262.jpg
1600
test_images/pm-00000

In [4]:
# gen url
count = 1
for i in range(count,297):
    name = "pm-{:010d}.jpg".format(i)
    print("https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Malariae/"+name)

https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Malariae/pm-0000000001.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Malariae/pm-0000000002.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Malariae/pm-0000000003.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Malariae/pm-0000000004.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Malariae/pm-0000000005.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Malariae/pm-0000000006.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Malariae/pm-0000000007.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Malariae/pm-0000000008.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Malariae/pm-0000000009.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Malariae/pm-0000000010.jpg
https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Malariae/pm-0000000011.jpg
https://ra

In [ ]:
count = 15250
for i in range(count,15367):
    name = "{:010d}".format(i)
    print(name)
    os.remove(os.path.join("images/",name+".jpg"))
    os.remove(os.path.join("labels/",name+".txt"))